GOAL:
    
Prepare function to merge tables together

In [16]:
from os.path import join, basename, splitext
from glob import glob
from dask import dataframe as dd
from dask import array as da
import dask
from matplotlib import rcParams
import pandas as pd
import dask
from collections import Counter
import pickle
import re


from deep_aqi import ROOT


pd.set_option('max_columns', 50)
pd.set_option('max_rows', 25)

In [17]:
INTERIM_DATA = join(ROOT, 'data', 'interim')
PROCESSED_DATA = join(ROOT, 'data', 'processed')

In [15]:
params = ['WIND', 'TEMP', 'PRESS', 'RH_DP', '42101', '88502', '42602', '44201', 'SPEC', '88101', '81102', '42401']

for param in params:
    print(param)
    files = glob(f'{INTERIM_DATA}/**.parquet', recursive=True)
    files = [file for file in files  if re.search(f'hourly_({param})', file)]
    tables = [dd.read_parquet(file) for file in files]
    df = dd.concat(tables)
    save_path = join(PROCESSED_DATA, f'{param}.parquet')
    df.to_parquet(save_path, engine='fastparquet', compression='SNAPPY')

WIND
TEMP
PRESS
RH_DP
42101
88502
42602
44201
SPEC
88101
81102
42401


In [11]:
len(files)

8

In [22]:
wind_path = join(PROCESSED_DATA, 'WIND.parquet')
temp_path = join(PROCESSED_DATA, 'TEMP.parquet')
press_path = join(PROCESSED_DATA, 'PRESS.parquet')
rhdp_path = join(PROCESSED_DATA, 'RH_DP.parquet')

wind = dd.read_parquet(wind_path)
temp = dd.read_parquet(temp_path)
press = dd.read_parquet(press_path)
rhdp = dd.read_parquet(rhdp_path)

In [32]:
weather = dd.merge(wind, temp, on=['SiteCode', 'LocalDate'])
weather = dd.merge(weather, press, on=['SiteCode', 'LocalDate'])
weather = dd.merge(weather, rhdp, on=['SiteCode', 'LocalDate'])

In [37]:
path_42101 = join(PROCESSED_DATA, '42101.parquet')
_42101 = dd.read_parquet(path_42101)

In [40]:
CO_42101 = dd.merge(weather, _42101, on=['SiteCode', 'LocalDate'], how='right')

In [ ]:
def create_dataset(param_name, weather_file=weather):
    file_path = join(PROCESSED_DATA, f'{param_name}')
    df = dd.read_parquet(file_path)
    merged = dd.merge(weather_file, df, on=['SiteCode', 'LocalDate'], how='right')
    return merged